In [4]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# basic imports
import tensorflow as tf 
import torch
import io
import numpy as np
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from keras.utils import dataset_utils
import matplotlib.pyplot as plt

import baseline_config

In [5]:
def paths_and_labels_to_dataset(image_paths,labels,num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    img_ds = path_ds.map(
        lambda path: tf.io.read_file(path), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    label_ds = dataset_utils.labels_to_dataset(
        labels, 
        'categorical', 
        num_classes)
    img_ds = tf.data.Dataset.zip((img_ds, label_ds))
    return img_ds

def create_dataset(subset):
    image_paths, labels, class_names = dataset_utils.index_directory(
            baseline_config.dataset_path + subset,
            labels="inferred",
            formats=('.pt'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=False)

    dataset = paths_and_labels_to_dataset(
        image_paths=image_paths,
        labels=labels,
        num_classes=len(class_names))
    
    return dataset, class_names

train_dataset, class_names = create_dataset('TRAIN/')
test_dataset, _            = create_dataset('TEST/')
validation_dataset, _      = create_dataset('VALIDATION/')
print("class names: ", class_names)

Found 12132 files belonging to 5 classes.
Found 505 files belonging to 5 classes.
Found 450 files belonging to 5 classes.
class names:  ['brant', 'jabwar', 'sheowl', 'spodov', 'wiltur']


In [6]:
def dataset_transforms(image,label):
    image = tf.io.parse_tensor(image, tf.float32)
    image = tf.expand_dims(image, -1)
    image = tf.repeat(image, 3, 2)
    return image,label

train_dataset_b = ( 
                  train_dataset
                  .shuffle(20000)
                  .map(dataset_transforms)
                  .batch(baseline_config.batch_size)
                  .cache()
                  .repeat()            
                )

validation_dataset_b = ( 
                  validation_dataset
                  .map(dataset_transforms)
                  .batch(baseline_config.batch_size)
                  .cache()
                )

test_dataset_b = ( 
                  test_dataset
                  .map(dataset_transforms)
                  .batch(baseline_config.batch_size)
                  .cache()
                )

In [7]:
for item,lbl in train_dataset_b.take(1):
    print(item.shape, lbl.shape)

(8, 313, 128, 3) (8, 5)


In [9]:
# build a really simple classification model using a pre-training Efficientnet V2
model = keras.Sequential(
    [
        # use the model as a feature generator only
        # need to resize here, as the efficientnet_v2_imagenet21k_s model expects it
        tf.keras.layers.InputLayer(input_shape=(313,128,3)),
        tf.keras.layers.Resizing(299, 299, interpolation="lanczos5", crop_to_aspect_ratio=False),
        
        # downloaded from: https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2
        # downloaded from: https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5
        hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=False),        

        # add the classification layer here       
        layers.Flatten(),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.60),
        layers.Dense(len(class_names), activation=None),
    ]
)
# need to tell the model what the input shape is
model.build([None, 299, 299, 3])

# show the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 299, 299, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 21,868,517
Trainable params: 65,733
Non-tr

In [10]:
# the form_logits means the loss function has the 'softmax' buillt in.  This approach is numerically more stable
# than including the softmax activation on the last layer of the classifier
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=baseline_config.learning_rate), 
              metrics=["accuracy"],
              )

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

model.fit(train_dataset_b, 
          validation_data=validation_dataset_b,
          steps_per_epoch=250,
          callbacks=[model_checkpoint_callback],
          epochs=baseline_config.max_epoch)

Epoch 1/100000
250/250 [==============================] - ETA: 0s - loss: 1.5997 - accuracy: 0.2730
Epoch 1: val_loss improved from inf to 1.58988, saving model to checkpoints/
250/250 [==============================] - 203s 766ms/step - loss: 1.5997 - accuracy: 0.2730 - val_loss: 1.5899 - val_accuracy: 0.2600
Epoch 2/100000
250/250 [==============================] - ETA: 0s - loss: 1.5540 - accuracy: 0.2600
Epoch 2: val_loss improved from 1.58988 to 1.54265, saving model to checkpoints/
250/250 [==============================] - 238s 949ms/step - loss: 1.5540 - accuracy: 0.2600 - val_loss: 1.5426 - val_accuracy: 0.2889
Epoch 3/100000
250/250 [==============================] - ETA: 0s - loss: 1.5287 - accuracy: 0.2630
Epoch 3: val_loss did not improve from 1.54265
250/250 [==============================] - 195s 779ms/step - loss: 1.5287 - accuracy: 0.2630 - val_loss: 1.5536 - val_accuracy: 0.2889
Epoch 4/100000
250/250 [==============================] - ETA: 0s - loss: 1.5275 - accurac